In [ ]:
import numpy as np
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
import os

def load_image(image_path):
    try:
        print(f"Trying to load image from: {image_path}")
        if not os.path.exists(image_path):
            print(f"File does not exist: {image_path}")
            return None
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_path}")
            return None
        image = cv2.resize(image, (224, 224))
        image = image.astype('float32') / 255.0
        return image
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

def prepare_dataset(image_paths, labels):
    X, y = [], []
    for image_path, label in zip(image_paths, labels):
        image = load_image(image_path)
        if image is None:
            continue
        X.append(image)
        y.append(label)

    print(f"X length: {len(X)}")
    print(f"y length: {len(y)}")
    print(f"Sample X: {X[0] if X else 'None'}")
    print(f"Sample y: {y[0] if y else 'None'}")

    X = np.array(X)
    y = np.array(y)

    lb = LabelBinarizer()
    y = lb.fit_transform(y)

    return train_test_split(X, y, test_size=0.2, random_state=42)

def create_model():
    model = Sequential()
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in vgg16.layers:
        layer.trainable = False
    model.add(vgg16)
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

image_paths = ["image1.jpg", "image2.jpg"]
labels = ["real", "fake"]

X_train, X_test, y_train, y_test = prepare_dataset(image_paths, labels)

if len(X_train) == 0 or len(X_test) == 0:
    print("No data to train or test. Please check the image paths and data preparation.")
else:
    model = create_model()
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)
    model.save('deepfake_detection_model_images.h5')

    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
